In [133]:
from itertools import compress, permutations
from collections import OrderedDict, Counter
from operator import itemgetter
import pandas as pd
import numpy as np
import random
import spacy

# Support 
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import datetime
import requests
import os, os.path
import time

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Display
from IPython.core.interactiveshell import InteractiveShell

# Spacy data:
nlp = spacy.load('en_core_web_sm')

# I/O
DIR = "HarvardVoc/"
load_file = 'harvard_words_only.csv'
save_file = 'harvard_words_only_tagged.csv'

hrvrd_old = pd.read_csv(DIR + load_file)


# Preparing dataframe
hrvrd_old = hrvrd_old.iloc[:, [j for j, c in enumerate(hrvrd_old.columns) if j != 0]]
hrvrd_old = hrvrd_old[pd.notnull(hrvrd_old['Entry'])].reset_index(drop=True)
hrvrd_old['Entry'] = hrvrd_old['Entry'].astype(str)

# Sum and drop duplicate rows
#df['Total'] = df.groupby(['Fullname', 'Zip'])['Amount'].transform('sum')
#new_df = df.drop_duplicates(subset=['Fullname', 'Zip'])

# Map and tag words
spacy_data = OrderedDict({'Entry': hs['Entry'], 'lemma': [], 'pos' : [], 'tag' : [], 'ent_type' : []})

for doc in nlp.pipe(hrvrd_old['Entry'].astype('unicode').values, batch_size=50, n_threads=4):
    if doc.is_parsed:
        spacy_data['lemma'].append([n.lemma_ for n in doc][0])
        spacy_data['pos'].append([n.pos_ for n in doc][0])
        spacy_data['tag'].append([n.tag_ for n in doc][0])
        spacy_data['ent_type'].append([n.ent_type_ for n in doc][0])
    else:
        spacy_data['lemma'].append(None)     #  Ab
        spacy_data['pos'].append(None)
        spacy_data['tag'].append(None)
        spacy_data['ent_type'].append(None)
        
# DF with taggeds words, out of two dfs
df_spacy_tags = pd.DataFrame.from_dict(spacy_data)
hrvrd_tags_df = pd.concat([df_spacy_tags, hrvrd_old.iloc[:,1:-2]], axis=1)

hrvrd_tags_df.to_csv(DIR + save_file, index=False)


print("DF after tagging:")
display(hrvrd_old.head(5))

DF after tagging:


,Entry,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,Weak,...,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othrtags,Defined
0,a,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,DET ART,| article: Indefinite singular article--some o...
1,abandon,False,True,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,SUPV,|
2,abandonment,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,Noun,|
3,abate,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,SUPV,|
4,abatement,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,Noun,NaN


In [ ]:
# Myster nan value in harvard database happens to be...

# # Also I located strange nan value(one) in Entry colum at 3899 idx. I cound and should have drop, 
# # Bu t i needend to check it just to sleap calm

# Entry_nan_row_idx = np.where(hrvrd_old['Entry'].isna()) # 
# file = 'HarvardVoc/HarvardVoc_InquirerAugmented.csv'
# xd = hrvrd_old.loc[3899]
# xd[xd == True] # columns to find it as index could have changed

# HarvardDF_raw = pd.read_csv(file)
# HarvardDF_raw.loc[3889:3909, ['Entry','Negativ', 'Ngtv', 'Vice', 'EVAL', 'RcEthic', 'RcTot']]

# # Result of investibation: It was, most certanlty on Harvard side. 
# # There is 0 (zero) value in entry idx 3900, and it happens to be F-letter part of 
# # database (alphabeticly speaking), so it's rather not O - my bet is: uts normal mistake or error 
# # in srcript used by Harcard during db construction. Anyway, i will check my scripts again.
# # Also, i feel more than free to  drop, if better idea comes up, source values are always avaliable in csv.

# x = HarvardDF_raw.loc[3900, ['Entry']]
# # Out: 0
# (int(x[0]) + 3 == 3)
# # Out: True
# (int("O") + 3 != 3) # Sanity check
# # Out: ValueError